In [16]:
import pandas as pd
import numpy as np

In [17]:
experiments_list = [
    # 'dim_multimodalmodel_wavlmlarge_robertalarge_lazyload_lr1e4_cka',
    # 'dim_multimodalmodel_wav2vec2_xls_2b_robertalarge_lazyload_lr1e5_cka',
    # "dim_multimodalmodel_wavlmlarge_robertalarge_lazyload_lr1e4_mse",
    "dim_multimodalmodel_wavlmlarge_robertalarge_lazyload_lr1e4",
    "dim_multimodalmodel_hubert_xlarge_robertalarge_lazyload_lr1e5",
    "dim_multimodalmodel_whisperlarge_robertalarge_lazyload_lr1e5"
]

dfs = []

for exp in experiments_list:

    dfs.append(pd.read_csv(f'../experiments/{exp}/results/test.csv'))
# preds.head()

In [18]:
# combined_df.shape, dfs[0].shape

In [19]:
combined_df = pd.concat(dfs, ignore_index=True)

In [20]:
preds = combined_df.groupby('Filename').agg({'EmoAct_pred': np.mean,'EmoVal_pred':np.mean,'EmoDom_pred':np.mean}).reset_index()

/tmp/ipykernel_1817001/3698676199.py:1: FutureWarning: The provided callable <function mean at 0x7b923d920700> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'mean' instead.
  preds = combined_df.groupby('Filename').agg({'EmoAct_pred': np.mean,'EmoVal_pred':np.mean,'EmoDom_pred':np.mean}).reset_index()
/tmp/ipykernel_1817001/3698676199.py:1: FutureWarning: The provided callable <function mean at 0x7b923d920700> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'mean' instead.
  preds = combined_df.groupby('Filename').agg({'EmoAct_pred': np.mean,'EmoVal_pred':np.mean,'EmoDom_pred':np.mean}).reset_index()


In [21]:
preds.head()

,Filename,EmoAct_pred,EmoVal_pred,EmoDom_pred
0,MSP-PODCAST_test3_0001.wav,2.269758,2.926405,2.597856
1,MSP-PODCAST_test3_0002.wav,3.871503,3.100317,4.102466
2,MSP-PODCAST_test3_0003.wav,1.748898,2.791306,2.299717
3,MSP-PODCAST_test3_0004.wav,3.142224,2.465293,3.528655
4,MSP-PODCAST_test3_0005.wav,2.999203,3.599456,3.421839


In [22]:
preds['FileName'] = preds['Filename']
preds['EmoAct'] = preds['EmoAct_pred']
preds['EmoVal'] = preds['EmoVal_pred']
preds['EmoDom'] = preds['EmoDom_pred']

columns = ['FileName', 'EmoAct', 'EmoVal', 'EmoDom']

In [23]:
preds = preds.sort_values(by='FileName').reset_index(drop = True)
preds[columns].to_csv("../submissions/dim_bimodal_ensemble3.csv", index=False)

In [29]:
df = pd.read_csv('/workspace/lucas.ueda/interspeech_ser/data/Labels/labels_consensus.csv')
df.head()

,FileName,EmoClass,EmoAct,EmoVal,EmoDom,SpkrID,Gender,Split_Set
0,MSP-PODCAST_0002_0033.wav,N,4.8,4.2,5.4,127,Female,Development
1,MSP-PODCAST_0002_0039.wav,N,4.0,4.2,4.2,127,Female,Development
2,MSP-PODCAST_0002_0051.wav,N,4.0,4.2,4.2,127,Female,Development
3,MSP-PODCAST_0002_0059.wav,X,4.0,3.8,4.0,128,Female,Development
4,MSP-PODCAST_0002_0061.wav,F,3.4,2.8,4.2,128,Female,Development


In [30]:
preds['FileName'] = preds['Filename']
preds = preds.merge(df[['FileName','EmoAct','EmoVal','EmoDom','Gender','SpkrID']], how='left', on = 'FileName')

In [31]:
# preds['EmoClass'].isnull().sum()

In [32]:
preds.head()

,Filename,EmoAct_pred,EmoVal_pred,EmoDom_pred,FileName,EmoAct,EmoVal,EmoDom,Gender,SpkrID
0,MSP-PODCAST_0002_0033.wav,3.969050,2.887507,3.883964,MSP-PODCAST_0002_0033.wav,4.8,4.2,5.4,Female,127
1,MSP-PODCAST_0002_0039.wav,3.851024,3.302966,3.826526,MSP-PODCAST_0002_0039.wav,4.0,4.2,4.2,Female,127
2,MSP-PODCAST_0002_0051.wav,3.477084,3.643494,3.642449,MSP-PODCAST_0002_0051.wav,4.0,4.2,4.2,Female,127
3,MSP-PODCAST_0002_0059.wav,3.752211,3.498477,3.744045,MSP-PODCAST_0002_0059.wav,4.0,3.8,4.0,Female,128
4,MSP-PODCAST_0002_0061.wav,3.706166,3.301337,3.757827,MSP-PODCAST_0002_0061.wav,3.4,2.8,4.2,Female,128


In [33]:
c = CCC_loss(preds[['EmoAct_pred','EmoVal_pred','EmoDom_pred']].values,preds[['EmoAct','EmoVal','EmoDom']].values)

In [34]:
c[0].item(), c[1].item(), c[2].item()

(0.6491772532463074, 0.6893381476402283, 0.5675678253173828)

In [35]:
# Average F1-score using bootstrap
act = []
val = []
dom = []

for i in range(100):
    preds_subset = preds.sample(1000).reset_index(drop=True)
    c = CCC_loss(preds_subset[['EmoAct_pred','EmoVal_pred','EmoDom_pred']].values,preds_subset[['EmoAct','EmoVal','EmoDom']].values)

    act.append(c[0].item())
    val.append(c[1].item())
    dom.append(c[2].item())

In [36]:
np.mean(act), np.mean(val), np.mean(dom)

(0.6512376266717911, 0.6896158766746521, 0.5711691707372666)

In [37]:
np.min(act), np.min(val), np.min(dom)

(0.5960695147514343, 0.6350388526916504, 0.5166829824447632)

In [38]:
np.max(act), np.max(val), np.max(dom)

(0.7011719346046448, 0.7274823784828186, 0.6214733123779297)

In [39]:
preds_subset = preds[preds['Gender'] == 'Female']
c = CCC_loss(preds_subset[['EmoAct_pred','EmoVal_pred','EmoDom_pred']].values,preds_subset[['EmoAct','EmoVal','EmoDom']].values)
c

tensor([0.5739, 0.6793, 0.4857])

In [40]:
preds_subset = preds[preds['Gender'] == 'Male']
c = CCC_loss(preds_subset[['EmoAct_pred','EmoVal_pred','EmoDom_pred']].values,preds_subset[['EmoAct','EmoVal','EmoDom']].values)
c

tensor([0.7239, 0.6932, 0.6665])